I have imported these libs to preprocess data.

In [ ]:
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('stopwords')

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

I am using Donal Trump tweets dataset. Importing the dataset using pandas and then extracting the required column then converting this dataframe to a python list



In [ ]:
import pandas as pd
df=pd.read_csv("realdonaldtrump.csv")
documents=df["content"]
documents=documents.to_list()

I have implemented function called cleaner this functions takes ndarray (numpy array) and then it removes urls, punctuations, single chars, stopswords and then lemmatize the word to its root form.

In [ ]:
stemmer=WordNetLemmatizer()
def cleaner(ndarr):
  tweets=ndarr.copy()
  pattern=r'(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))';
  for i in range(len(tweets)):
      tweet=tweets[i]
      tweet=re.sub(pattern," ",tweet)
      tweet=re.sub(r"\W"," ",tweet)
      tweet=re.sub(re.escape(string.punctuation),"",tweet) 
      tweet=re.sub(r"\s+[a-zA-Z]\s+"," ",tweet) 
      tweet=re.sub(r"\^[a-zA-Z]\s+"," ",tweet)  
      tweet=re.sub(r"\s+"," ",tweet,flags=re.I)
      tweet=tweet.lower()
      tweet=tweet.split()
      tweet=[stemmer.lemmatize(word) for word in tweet]
      tweet=[word for word in tweet if word not in stopwords.words("english")]

      tweet=" ".join(tweet)
      tweets[i]=tweet
  return tweets

After cleaning the tweets I have extracted the features using count vectorizer
Some feature names are shown below:

In [ ]:
import numpy as np
documents=np.array(documents)
documents=cleaner(documents)
douments=documents.tolist()
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
no_features = 1000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()
tf_feature_names

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '000',
 '10',
 '100',
 '11',
 '12',
 '13',
 '15',
 '16',
 '18',
 '20',
 '2012',
 '2016',
 '2020',
 '25',
 '2nd',
 '30',
 '3rd',
 '50',
 'abc',
 'able',
 'absolutely',
 'act',
 'action',
 'actually',
 'ad',
 'adam',
 'address',
 'administration',
 'advice',
 'afraid',
 'african',
 'agenda',
 'agent',
 'ago',
 'agree',
 'agreed',
 'ahead',
 'air',
 'al',
 'alabama',
 'alexsalmond',
 'allow',
 'allowed',
 'amazing',
 'amendment',
 'america',
 'american',
 'andrew',
 'angry',
 'announce',
 'announced',
 'announcement',
 'answer',
 'anti',
 'anymore',
 'apologize',
 'apprentice',
 'apprenticenbc',
 'approval',
 'area',
 'arizona',
 'art',
 'article',
 'ask',
 'asked',
 'asking',
 'attack',
 'attorney',
 'award',
 'away',
 'awesome',
 'bad',
 'badly',
 'bank',
 'barackobama',
 'based',
 'beach',
 'beat',
 'beautiful',
 'begin',
 'beginning',
 'bei',
 'believe',
 'bernie',
 'best',
 'better',
 'biased',
 'biden',
 'big',
 'bigger',
 'biggest',
 'billion',
 'billionaire',
 'billmaher',

Now importing LDA from sklearn lib.
We have configured 10 topics and 10 maximum iterations. And using fit function to train data

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, max_iter=10, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

I have implemented this get_topics_output function this function takes model, feature names that we have extracted earlier and no of top words. The give us the topics that are modeled by LDA.

In [ ]:
def get_topics_output(model, feature_names, no_top_words):
    for index, topic in enumerate(model.components_):
        print (f"Topic {index}:"," ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
no_top_words = 10
get_topics_output(lda, tf_feature_names, no_top_words)

Topic 0: trump thank big donald clinton hillary soon north doe administration
Topic 1: make wa american great know deal house say history think
Topic 2: new good news way foxnews poll great money man real
Topic 3: realdonaldtrump trump2016 2016 high apprentice start great crowd apprenticenbc week
Topic 4: people ha america job need border fake want president republican
Topic 5: country love going look year right looking party forward like
Topic 6: great time thanks today night wa foxandfriends watch cnn wall
Topic 7: vote run day great trump business national working nice 000
Topic 8: president democrat state obama wa medium work united election military
Topic 9: true great tax trump congratulation usa mexico win cut john
